In [1]:
from math import log
import operator

In [2]:
def createDataset():
    dataset=[[1,1,'yes'],
             [1,1,'yes'],
             [1,0,'no'],
             [0,1,'no'],
             [0,1,'no'],]
    labels=['no surfacing','flippers']
    return dataset,labels

X_train,y_train=createDataset()
labels=y_train.copy()

> 计算熵,entrop

$$\sum_{k=0}^{K} -p_{k}log_{2}p_{k}$$

In [3]:
def calcShannonEnt(dataset):
    numItems=len(dataset)
    labelCounts={}
    for itemset in dataset:
        currentLabel=itemset[-1]
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numItems
        shannonEnt-=prob*log(prob,2)
    return shannonEnt

calcShannonEnt(X_train)

0.9709505944546686

In [4]:
# splitDataSet
# 在dataSet中取出第axis个元素.并判断这个元素值和value是否相等.
# 如果相等,这个vector取出来,并把当前的axis的这个元素从这个vector中删除.
# 并以此重新组成一个retDataSet,也就是剩下的Data集.

def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]     # 扣掉这个axis描述的内容.
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)   # 添加到retDataSet中
    return retDataSet

T=splitDataSet(X_train,0,1)
print(T)
T=splitDataSet(X_train,0,0)
print(T)
T=splitDataSet(X_train,1,0)
print(T)
T=splitDataSet(X_train,1,1)
print(T)
T=splitDataSet(X_train,2,'yes')
print(T)
T=splitDataSet(X_train,2,'no')
print(T)

[[1, 'yes'], [1, 'yes'], [0, 'no']]
[[1, 'no'], [1, 'no']]
[[1, 'no']]
[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]
[[1, 1], [1, 1]]
[[1, 0], [0, 1], [0, 1]]


In [5]:
#len(X_train[0])
X_train[0]

[1, 1, 'yes']

#### g(D,A)

1.信息增益
$$g(D,A)=H(D)-H(D|A)$$

2.信息熵
$$H(D)=\sum_{k=1}^{K}\frac{|C_k|}{N}log\frac{|C_k|}{N}$$

3.H(D|A)
$$H(D|X^{(A)}=a_i)=\sum_{k=1}^{K}-(\frac{N_{ik}}{N_i}log\frac{N_{ik}}{N_i})$$

$$H(D|A)=\sum_{i=1}^{N}P_A(X^{A})H(D|X^{A}=a_i)=\sum_{i=1}^{N}\frac{N_i}{N}*\sum_{k=1}^{K}-(\frac{N_{ik}}{N_i}log\frac{N_{ik}}{N_i})$$

In [6]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures=len(dataSet[0])-1
    baseEntropy=calcShannonEnt(dataSet)
    bestInfoGain=0.0
    bestFeature=-1
    for i in range(numFeatures): # 一共2个feature. 依次取第一个和第二个feature.
        featList=[example[i] for example in dataSet] # 所有的dataSet中取出第一个
        #print("featList",featList)
        uniqueVals=set(featList) # 这个应该是种类的个数. 比如第一个feature有两类,"1"和"0"
        newEntropy=0.0
        for value in uniqueVals:
            subDataSet=splitDataSet(dataSet,i,value)
            #print("subDataSet",subDataSet)
            #print("len(subDataSet)",len(subDataSet))
            #print("len(dataSet)",len(dataSet))
            prob=len(subDataSet)/float(len(dataSet)) # 算的 Ni/N
            newEntropy+=prob*calcShannonEnt(subDataSet) # 算的H(D|A)
        infoGain = baseEntropy-newEntropy 
        if(infoGain>bestInfoGain): # 找到最大的信息熵对应的feature
            bestInfoGain=infoGain
            bestFeature=i
    return bestFeature

chooseBestFeatureToSplit(X_train)

0

In [7]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote]=0
        classCount[vote]+=1
    sortedClassCount=sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]
    print("classList",classList)
    
    # 两个结束条件
    if classList.count(classList[0])==len(classList): # 该叶子节点只有一个成员了.就退出
        return classList[0]
    
    if len(dataSet[0])==1:
        return majorityCnt(classList) # 如果某个子集中都是同一类的,就不用再分了.
    
    bestFeat=chooseBestFeatureToSplit(dataSet) # 算出当前dataSet中最大熵的feature.
    bestFeatLabel=labels[bestFeat]
    #print("bestFeat",bestFeat)
    #print("bestFeatLabel",bestFeatLabel)
    myTree={bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues=[example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:] # 拷贝
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    
    return myTree

        
#createTree(X_train[:-1],X_train[-1])
mytree=createTree(X_train,y_train)

classList ['yes', 'yes', 'no', 'no', 'no']
classList ['no', 'no']
classList ['yes', 'yes', 'no']
classList ['no']
classList ['yes', 'yes']


In [10]:
def classify(inputTree,featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

print(classify(mytree,labels,[1,0]))
print(classify(mytree,labels,[0,1]))
print(classify(mytree,labels,[1,1]))

no
no
yes
